In [ ]:
import json
from pathlib import Path

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [ ]:
log_directory = Path("/Users/andreas/workspace/thesis-code/exp-backup/privacy")
result_files = [path for path in log_directory.iterdir() if path.suffix == ".csv" and "results_" in path.name]

dfs = []
hyperparams = []

for f in result_files:
    print(f"Loading results file \"{f.name}\"")
    dfs.append(pd.read_csv(f))

    parameters = ["devices", "epochs", "local_epochs", "local_batch_size", "clients_per_round",
                  "l2_norm_clip", "noise_multiplier", "local_dp", "start_time"]
    parameter_values = f.stem.lstrip("results_").split("_")
    assert len(parameter_values) == len(parameters)
    hyperparams.append(
        {key: value for (key, value) in zip(parameters, parameter_values)}
    )
experiments = list(zip(hyperparams, dfs))

In [ ]:
# Test Loss Plot
fig, ax = plt.subplots()
for params, df in experiments:
    df.plot(x="epoch", y="test_loss", ax=ax, label=f'DP: {params["local_dp"]} L2-Clip: {params["l2_norm_clip"]} Noise-Mult:{params["noise_multiplier"]}')
fig.savefig(log_directory / "test_loss.pdf")

In [ ]:
# Test Accuracy Plot
fig, ax = plt.subplots(figsize=(20, 10))
for params, df in experiments:
    df.plot(x="epoch", y="test_accuracy", ax=ax, label=f'DP: {params["local_dp"]} L2-Clip: {params["l2_norm_clip"]} Noise-Mult:{params["noise_multiplier"]}')
fig.savefig(log_directory / "test_accuracy.pdf")

In [ ]:
dfs[0]

In [ ]:
# Eps Guarantees
privacy_dfs = []
for params, df in [exp for exp in experiments if exp[0]["local_dp"] == "True"]:
    df = df.copy()
    df["privacy_params"] =  f'{params["l2_norm_clip"]}_{params["noise_multiplier"]}'
    df["privacy_guarantees"] = df.privacy_guarantees.map(lambda x: x.replace("'", "")).map(json.loads)
    eps_per_round_and_client = df["privacy_guarantees"][0][0]['eps']
    delta_per_round_and_client = df["privacy_guarantees"][0][0]['delta']
    df["eps_per_round_and_client"] = eps_per_round_and_client
    df["delta_per_round_and_client"] = delta_per_round_and_client
    df["eps_delta_per_round_and_client"] = f"({eps_per_round_and_client}, {delta_per_round_and_client})"
    privacy_dfs.append(df)

combined_df = pd.concat(privacy_dfs)
fig, ax = plt.subplots()
sns.barplot(x="privacy_params", y="eps_per_round_and_client", data=combined_df, ax=ax)
    #df.plot(x="epoch", y="test_accuracy", ax=ax, label=f'Clip: {params["l2_norm_clip"]} Noise:{params["noise_multiplier"]}')
#fig.savefig(log_directory / "test_accuracy.pdf")

In [ ]:
eps_per_round_and_client, delta_per_round_and_client